### Imports

In [18]:
import keras
import os
import cv2
import dlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import metrics, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Input, Dense, Conv2D, Flatten, GlobalAveragePooling2D, MaxPooling2D, Layer, concatenate, Activation, Add
from sklearn.model_selection import train_test_split
from keras.applications import VGG16, MobileNetV2
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from tqdm import tqdm
from mtcnn.mtcnn import MTCNN
import face_recognition
from deepface import DeepFace
from keras.utils import custom_object_scope

### Prepare data

-   data contains NaN values on normalized cols => complete the dataset

In [2]:
dataset_folder = './PoG Dataset' # options: GazeDataset (face-only images) || PoG Dataset (full images)

In [3]:
def normalize_labels(df):
    df['x_norm'] = df['x_pixels'] / df['width_pixels']
    df['y_norm'] = df['y_pixels'] / df['height_pixels']
    return df

In [4]:
def denormalize(y, idx, test_generator, test_meta):
    filename = test_generator.filenames[idx]
    screen = test_meta.loc[test_meta['filename'] == filename, ['width_mm', 'height_mm']]
    width = screen['width_mm'].values[0]
    height = screen['height_mm'].values[0]
    y[0][0] *= width
    y[0][1] *= height
    return y

In [5]:
train_meta, validation_meta, test_meta = normalize_labels(pd.read_csv('./train.csv')), normalize_labels(pd.read_csv('./validation.csv')), normalize_labels(pd.read_csv('./test.csv'))

In [7]:
train_meta

,filename,x_pixels,y_pixels,width_pixels,height_pixels,width_mm,height_mm,human_pc_distance(cm),x_norm,y_norm
0,imp1.jpg,624.0,90.0,1920,1080,344.443871,193.749677,NaN,0.325000,0.083333
1,imp2.jpg,553.0,206.0,1920,1080,344.443871,193.749677,NaN,0.288021,0.190741
2,imp3.jpg,277.0,231.0,1920,1080,344.443871,193.749677,NaN,0.144271,0.213889
3,imp4.jpg,1405.0,254.0,1920,1080,344.443871,193.749677,NaN,0.731771,0.235185
4,imp5.jpg,1446.0,373.0,1920,1080,344.443871,193.749677,NaN,0.753125,0.345370
...,...,...,...,...,...,...,...,...,...,...
16966,PTC768_1187.jpg,768.0,1187.0,2496,1664,317.000000,212.000000,50.0,0.307692,0.713341
16967,PTC771_1347.jpg,771.0,1347.0,2496,1664,317.000000,212.000000,50.0,0.308894,0.809495
16968,PTC776_830.jpg,776.0,830.0,2496,1664,317.000000,212.000000,50.0,0.310897,0.498798
16969,PTC778_467.jpg,778.0,467.0,2496,1664,317.000000,212.000000,50.0,0.311699,0.280649


In [8]:
validation_meta

,filename,x_pixels,y_pixels,width_pixels,height_pixels,width_mm,height_mm,human_pc_distance(cm),x_norm,y_norm
0,imp501.jpg,1165.0,288.0,1920,1080,344.443871,193.749677,NaN,0.606771,0.266667
1,imp502.jpg,1175.0,376.0,1920,1080,344.443871,193.749677,NaN,0.611979,0.348148
2,imp503.jpg,1260.0,380.0,1920,1080,344.443871,193.749677,NaN,0.656250,0.351852
3,imp504.jpg,1333.0,420.0,1920,1080,344.443871,193.749677,NaN,0.694271,0.388889
4,imp505.jpg,1383.0,495.0,1920,1080,344.443871,193.749677,NaN,0.720313,0.458333
...,...,...,...,...,...,...,...,...,...,...
1645,PTC768_1187.jpg,768.0,1187.0,2496,1664,317.000000,212.000000,50.0,0.307692,0.713341
1646,PTC771_1347.jpg,771.0,1347.0,2496,1664,317.000000,212.000000,50.0,0.308894,0.809495
1647,PTC776_830.jpg,776.0,830.0,2496,1664,317.000000,212.000000,50.0,0.310897,0.498798
1648,PTC778_467.jpg,778.0,467.0,2496,1664,317.000000,212.000000,50.0,0.311699,0.280649


In [9]:
test_meta

,filename,x_pixels,y_pixels,width_pixels,height_pixels,width_mm,height_mm,human_pc_distance(cm),x_norm,y_norm
0,imp551.jpg,1077.0,378.0,1920,1080,344.443871,193.749677,NaN,0.560937,0.350000
1,imp552.jpg,1073.0,285.0,1920,1080,344.443871,193.749677,NaN,0.558854,0.263889
2,imp553.jpg,1181.0,325.0,1920,1080,344.443871,193.749677,NaN,0.615104,0.300926
3,imp554.jpg,969.0,360.0,1920,1080,344.443871,193.749677,NaN,0.504687,0.333333
4,imp555.jpg,976.0,275.0,1920,1080,344.443871,193.749677,NaN,0.508333,0.254630
...,...,...,...,...,...,...,...,...,...,...
1645,PTC978_739.jpg,978.0,739.0,2496,1664,317.000000,212.000000,50.0,0.391827,0.444111
1646,PTC981_499.jpg,981.0,499.0,2496,1664,317.000000,212.000000,50.0,0.393029,0.299880
1647,PTC981_851.jpg,981.0,851.0,2496,1664,317.000000,212.000000,50.0,0.393029,0.511418
1648,PTC99_531.jpg,99.0,531.0,2496,1664,317.000000,212.000000,50.0,0.039663,0.319111


In [19]:
train_datagen, val_datagen, test_datagen = ImageDataGenerator(rescale=1./255), ImageDataGenerator(rescale=1./255), ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
                                        train_meta, 
                                        directory=dataset_folder, 
                                        x_col='filename', 
                                        y_col=['x_norm', 'y_norm'], 
                                        class_mode='raw',
                                        batch_size=64, 
                                        target_size=(224, 224), 
                                        color_mode='rgb', 
                                        shuffle=True)
validation_generator = val_datagen.flow_from_dataframe(
                                        validation_meta, 
                                        directory=dataset_folder, 
                                        x_col='filename', 
                                        y_col=['x_norm', 'y_norm'], 
                                        class_mode='raw',
                                        batch_size=64, 
                                        target_size=(224, 224), 
                                        color_mode='rgb', 
                                        shuffle=False)
test_generator = test_datagen.flow_from_dataframe(
                                        test_meta, 
                                        directory=dataset_folder, 
                                        x_col='filename', 
                                        y_col=['x_norm', 'y_norm'], 
                                        class_mode='raw',
                                        batch_size=1, 
                                        target_size=(224, 224), 
                                        color_mode='rgb', 
                                        shuffle=False)

Found 16971 validated image filenames.
Found 1650 validated image filenames.
Found 1650 validated image filenames.


### Training & testing

- final error is in mm

#### Test saved models

In [20]:
with custom_object_scope({'RandomCropAugmentation': RandomCropAugmentation}):
    model = load_model('./CVGazeDetectionModels/PoGD-Maxi-RCrop-4,932.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 random_crop_augmentation (Rand  (None, 224, 224, 3)  0          ['input_1[0][0]']                
 omCropAugmentation)                                                                              
                                                                                                  
 conv2d_1 (Conv2D)              (None, 224, 224, 16  64          ['random_crop_augmentation[0][0]'
                                )                                ]                            

In [22]:
score = model.evaluate(test_generator)

print('Test loss:', score)

1650/1650 [==============================] - 69s 42ms/step - loss: 0.0475
Test loss: 0.04750590771436691


In [23]:
test_loss = 0
for i in tqdm(range(len(test_generator))):
    x_test, y_test = test_generator[i]
    y_pred = model.predict(x_test, verbose=0)
    
    y_test_denorm = denormalize(y_test, i, test_generator, test_meta)
    y_pred_denorm = denormalize(y_pred, i, test_generator, test_meta)
    test_loss += K.sqrt(K.mean(K.square(y_test_denorm - y_pred_denorm)))

test_loss /= len(test_generator)
print(f'Test loss: {test_loss} mm.')

100%|██████████| 1650/1650 [06:13<00:00,  4.42it/s]

Test loss: 49.321030861116256 mm.


#### Simple CNN - 2,389,986 params

In [11]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(2))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[
                    metrics.MeanSquaredError(),
                    metrics.RootMeanSquaredError(),
                    metrics.MeanAbsoluteError(),
                    metrics.MeanAbsolutePercentageError(),
                    metrics.MeanSquaredLogarithmicError()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [ ]:
checkpoint = ModelCheckpoint('./checkpoints/SimpleCNN-{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.model', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

callbacks = [checkpoint, reduce_lr]

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, callbacks=callbacks, epochs=100)

In [14]:
model = load_model('./CVGazeDetectionModels/GazeD-SimpleCNN-3,465.model')

In [15]:
score = model.evaluate(test_generator)

print('Test loss:', score[0])

1650/1650 [==============================] - 39s 22ms/step - loss: 0.0268 - mean_squared_error: 0.0268 - root_mean_squared_error: 0.1636 - mean_absolute_error: 0.1161 - mean_absolute_percentage_error: 1726219.0000 - mean_squared_logarithmic_error: 0.0128
Test loss: 0.026773331686854362


In [17]:
test_loss = 0
for i in tqdm(range(len(test_generator))):
    x_test, y_test = test_generator[i]
    y_pred = model.predict(x_test, verbose=0)
    
    y_test_denorm = denormalize(y_test, i, test_generator, test_meta)
    y_pred_denorm = denormalize(y_pred, i, test_generator, test_meta)
    test_loss += K.sqrt(K.mean(K.square(y_test_denorm - y_pred_denorm)))

test_loss /= len(test_generator)
print(f'Test loss: {test_loss} mm.')

100%|██████████| 1650/1650 [03:41<00:00,  7.44it/s]

Test loss: 34.65047158620538 mm.


In [ ]:
model.save('SimpleCNN.h5')

#### Gaze models

In [17]:
class RandomCropAugmentation(Layer):
    def __init__(self, min_crop_ratio=0.1, max_crop_ratio=0.4):
        super().__init__()
        self.min_crop_ratio = min_crop_ratio
        self.max_crop_ratio = max_crop_ratio

    def call(self, inputs, training=None):
        if training and np.random.rand() < 0.5:
            input_shape = tf.shape(inputs)
            h, w = input_shape[1], input_shape[2]

            min_val = tf.cast(tf.cast(h, tf.float32) * self.min_crop_ratio, tf.int32)
            max_val = tf.cast(tf.cast(w, tf.float32) * self.max_crop_ratio, tf.int32)
            crop_height = tf.random.uniform(shape=[], minval=min_val, maxval=max_val, dtype=tf.int32)
            crop_width = tf.random.uniform(shape=[], minval=min_val, maxval=max_val, dtype=tf.int32)

            crop_top = tf.random.uniform(shape=[], minval=0, maxval=h-crop_height, dtype=tf.int32)
            crop_left = tf.random.uniform(shape=[], minval=0, maxval=w-crop_width, dtype=tf.int32)

            cropped_image = inputs[:, crop_top:crop_top+crop_height, crop_left:crop_left+crop_width, :]

            mask = tf.ones_like(cropped_image)
            padding = [[0, 0], [crop_top, h-crop_top-crop_height], [crop_left, w-crop_left-crop_width], [0, 0]]
            mask = tf.pad(mask, padding)
            cropped_image = tf.pad(cropped_image, padding)

            inputs = inputs * (1 - mask) + cropped_image * mask

        return inputs

#### GazeModelMaxi - 379,234 params

In [31]:
input_img = Input(shape=(224, 224, 3))
cropped_img = RandomCropAugmentation()(input_img)

skip = cropped_img
skip = Conv2D(32, kernel_size=3, padding='valid')(skip)
skip = MaxPooling2D(pool_size=4, strides=2)(skip)

x = Conv2D(16, kernel_size=1, padding="valid")(cropped_img)
x = Activation('relu')(x)
x = Conv2D(16, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(16, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)

x = Conv2D(32, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=1, padding="valid")(x)

x = MaxPooling2D()(x)

x = Add()([x, skip])
x = Activation('relu')(x)

skip = x
skip = Conv2D(64, kernel_size=3, strides=2, padding='valid')(skip)
skip = MaxPooling2D(pool_size=4, strides=2)(skip)

x = Conv2D(32, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)

x = MaxPooling2D()(x)

x = Conv2D(64, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(64, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(64, kernel_size=1, padding="valid")(x)

x = MaxPooling2D()(x)

x = Add()([x, skip])
x = Activation('relu')(x)

skip = x
skip = Conv2D(128, kernel_size=3, strides=2, padding='valid')(skip)
skip = MaxPooling2D(pool_size=4, strides=2)(skip)

x = Conv2D(64, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(64, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(64, kernel_size=1, padding="valid")(x)

x = MaxPooling2D()(x)

x = Conv2D(128, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(128, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(128, kernel_size=1, padding="valid")(x)

x = MaxPooling2D()(x)

x = Add()([x, skip])
x = Activation('relu')(x)

x = GlobalAveragePooling2D()(x)
output = Dense(2)(x)

model = Model(inputs=input_img, outputs=output)

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 random_crop_augmentation_4 (Ra  (None, 224, 224, 3)  0          ['input_8[0][0]']                
 ndomCropAugmentation)                                                                            
                                                                                                  
 conv2d_85 (Conv2D)             (None, 224, 224, 16  64          ['random_crop_augmentation_4[0][0
                                )                                ]']                        

In [32]:
checkpoint = ModelCheckpoint('./checkpoints/GazeMaxi-{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.model', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

callbacks = [checkpoint, reduce_lr]

In [33]:
history = model.fit(train_generator, validation_data=validation_generator, callbacks=callbacks, epochs=100)

Epoch 1/100
  7/266 [..............................] - ETA: 27:13 - loss: 0.1461

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(test_generator)

print('Test loss:', score)

In [ ]:
test_loss = 0
for i in tqdm(range(len(test_generator))):
    x_test, y_test = test_generator[i]
    y_pred = model.predict(x_test, verbose=0)
    
    y_test_denorm = denormalize(y_test, i)
    y_pred_denorm = denormalize(y_pred, i)
    test_loss += K.sqrt(K.mean(K.square(y_test_denorm - y_pred_denorm)))

test_loss /= len(test_generator)
print(f'Test loss: {test_loss} mm.')

In [ ]:
model.save('GazeMaxi.h5')

#### GazeModelMini - 87,586 params

In [ ]:
input_img = Input(shape=(224, 224, 3))
cropped_img = RandomCropAugmentation()(input_img)

skip = cropped_img
skip = Conv2D(32, kernel_size=3, padding='valid')(skip)
skip = MaxPooling2D(pool_size=4, strides=2)(skip)

x = Conv2D(16, kernel_size=1, padding="valid")(cropped_img)
x = Activation('relu')(x)
x = Conv2D(16, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(16, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)

x = Conv2D(32, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=1, padding="valid")(x)

x = MaxPooling2D()(x)

x = Add()([x, skip])
x = Activation('relu')(x)

skip = x
skip = Conv2D(64, kernel_size=3, strides=2, padding='valid')(skip)
skip = MaxPooling2D(pool_size=4, strides=2)(skip)

x = Conv2D(32, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(32, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)

x = MaxPooling2D()(x)

x = Conv2D(64, kernel_size=1, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(64, kernel_size=3, padding="valid")(x)
x = Activation('relu')(x)
x = Conv2D(64, kernel_size=1, padding="valid")(x)

x = MaxPooling2D()(x)

x = Add()([x, skip])
x = Activation('relu')(x)

x = GlobalAveragePooling2D()(x)
output = Dense(2)(x)

model = Model(inputs=input_img, outputs=output)

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
checkpoint = ModelCheckpoint('./checkpoints/GazeMini-{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.model', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

callbacks = [checkpoint, reduce_lr]

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, callbacks=callbacks, epochs=100)

In [ ]:
score = model.evaluate(test_generator)

print('Test loss:', score)

In [ ]:
test_loss = 0
for i in tqdm(range(len(test_generator))):
    x_test, y_test = test_generator[i]
    y_pred = model.predict(x_test, verbose=0)
    
    y_test_denorm = denormalize(y_test, i)
    y_pred_denorm = denormalize(y_pred, i)
    test_loss += K.sqrt(K.mean(K.square(y_test_denorm - y_pred_denorm)))

test_loss /= len(test_generator)
print(f'Test loss: {test_loss} mm.')

In [ ]:
model.save('GazeMini.h5')

### Retrieve faces from images

-   recommendation: should use DeepFace for all images

In [ ]:
# dlib  
detector = dlib.get_frontal_face_detector()

def crop_face(image_path, output_folder):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray, 1)

    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()

        face_cropped = image[y:y+h, x:x+w]
        if 0 in face_cropped.shape:
            continue
        face_resized = cv2.resize(face_cropped, (224, 224))

        base_name = os.path.basename(image_path)
        cv2.imwrite(os.path.join(output_folder, base_name), face_resized)

def process_images_in_folder(folder_path='./PoG Dataset', output_folder='./GazeDataset'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if os.path.isfile(image_path):
            if not os.path.isfile(os.path.join(output_folder, os.path.basename(image_path))):
                print(f'Processing image: {image_path}')
                crop_face(image_path, output_folder)

In [ ]:
# MTCNN
detector = MTCNN()

def crop_face(image_path, output_folder):
    image = cv2.imread(image_path)
    faces = detector.detect_faces(image)

    for face in faces:
        x, y, w, h = face['box']

        face_cropped = image[y:y+h, x:x+w]
        if 0 in face_cropped.shape:
            continue
        face_resized = cv2.resize(face_cropped, (224, 224))

        base_name = os.path.basename(image_path)
        cv2.imwrite(os.path.join(output_folder, base_name), face_resized)

def process_images_in_folder(folder_path='./PoG Dataset', output_folder='./GazeDataset'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if os.path.isfile(image_path):
            if not os.path.isfile(os.path.join(output_folder, os.path.basename(image_path))):
                print(f'Processing image: {image_path}')
                crop_face(image_path, output_folder)

In [ ]:
# OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def crop_face(image_path, output_folder):
    image = cv2.imread(image_path)
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.05, minNeighbors=3, minSize=(50, 50))

    for (x, y, w, h) in faces:
        face_cropped = image[y:y+h, x:x+w]
        if 0 in face_cropped.shape:
            continue
        face_resized = cv2.resize(face_cropped, (224, 224))

        base_name = os.path.basename(image_path)
        cv2.imwrite(os.path.join(output_folder, base_name), face_resized)

def process_images_in_folder(folder_path='./PoG Dataset', output_folder='./GazeDataset'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if os.path.isfile(image_path):
            if not os.path.isfile(os.path.join(output_folder, os.path.basename(image_path))):
                print(f'Processing image: {image_path}')
                crop_face(image_path, output_folder)

In [ ]:
# face_recognition
def crop_face(image_path, output_folder):
    image = face_recognition.load_image_file(image_path)
    face_locations = face_recognition.face_locations(image)

    for (top, right, bottom, left) in face_locations:
        face_cropped = image[top:bottom, left:right]
        if 0 in face_cropped.shape:
            continue
        face_resized = cv2.resize(face_cropped, (224, 224))

        base_name = os.path.basename(image_path)
        cv2.imwrite(os.path.join(output_folder, base_name), face_resized)

def process_images_in_folder(folder_path='./PoG Dataset', output_folder='./GazeDataset'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if os.path.isfile(image_path):
            if not os.path.isfile(os.path.join(output_folder, os.path.basename(image_path))):
                print(f'Processing image: {image_path}')
                crop_face(image_path, output_folder)


In [ ]:
# DeepFace - best face recognition method (no strange or undetected images)
def crop_face(image_path, output_folder):
    image = cv2.imread(image_path)
    detected_faces = DeepFace.extract_faces(image, 'yolov8', False)

    face = detected_faces[0]['facial_area']

    (x, y, w, h) = (face['x'], face['y'], face['w'], face['h'])
    face_cropped = image[y:y+h, x:x+w]
    face_resized = cv2.resize(face_cropped, (224, 224))

    base_name = os.path.basename(image_path)
    cv2.imwrite(os.path.join(output_folder, base_name), face_resized)

def process_images_in_folder(folder_path='./PoG Dataset', output_folder='./GazeDataset'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if os.path.isfile(image_path):
            if not os.path.isfile(os.path.join(output_folder, os.path.basename(image_path))):
                print(f'Processing image: {image_path}')
                crop_face(image_path, output_folder)


In [ ]:
process_images_in_folder()